# **Using Pandas with Google BigQuery**

A notebook on how to use Google Colab, BigQuery and Pandas.

#### **Dataset**
The following dataset is used:
https://archive.ics.uci.edu/dataset/352/online+retail

#### **Trouble shooting**

##### **Expected bytes, got a 'int' object**
Identify the pandas columns that generates this error and convert to a String

##### **User does not have bigquery.jobs.create permission in project**
Ensure the service account have the right role: "BigQuery User"

##### **User does not have permission to query table [...]**
Ensure the service account have the right role: "BigQuery Data Viewer"


In [1]:
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account

from google.colab import drive


In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [29]:
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/'

datafile = base_dir + '...' #TODO: Update with correct path for datafile

In [4]:
data = pd.read_excel(datafile)
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
data.drop(data[data['CustomerID'].isna()].index, inplace = True) # Removes all null values in Customer ID

In [7]:
data['CustomerID'] = data['CustomerID'].astype('Int64')

data['InvoiceNo'] = data['InvoiceNo'].astype('str') # fixes the following error: Expected bytes, got a 'int' object
data['StockCode'] = data['StockCode'].astype('str') # fixes the following error: Expected bytes, got a 'int' object

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  Int64         
 7   Country      406829 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 28.3+ MB



## **Setting up the BigQuery client**

In [25]:
project_id = "..." #TODO: update with the correct project id
dataset_id = "..." #TODO: update with the correct dataset id

credentials_jsonfile = "..." #TODO: Update this to the location of your service account json file

In [22]:
credentials = service_account.Credentials.from_service_account_file(
    base_dir + credentials_jsonfile,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

In [12]:
client = bigquery.Client(
    credentials = credentials,
    project = project_id #credentials.project_id
)

## **Create table in BigQuery**

In [27]:
table_name = "..." #TODO: update with the table name of your choice
table_id = "{dataset}.{table}".format(dataset = dataset_id, table = table_name)

In [14]:
data.to_gbq(
    table_id,
    project_id = project_id,
    if_exists="replace"  # fail / replace / append
  )

100%|██████████| 1/1 [00:00<00:00, 6223.00it/s]


## **Querying data with the GoogleSQL syntax**

* https://cloud.google.com/bigquery/docs/pandas-gbq-migration

In [28]:
sql = """
  SELECT *
  FROM {table}
 """.format(table=table_id) #OBS: table is case-sensitive

In [16]:
df_bigQuery = client.query(sql).to_dataframe()

In [17]:
df_bigQuery.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,571035,21238,RED RETROSPOT CUP,8,2011-10-13 12:50:00+00:00,0.85,12446,RSA
1,571035,21243,PINK POLKADOT PLATE,8,2011-10-13 12:50:00+00:00,1.69,12446,RSA
2,571035,23240,SET OF 4 KNICK KNACK TINS DOILY,6,2011-10-13 12:50:00+00:00,4.15,12446,RSA
3,571035,23209,LUNCH BAG VINTAGE DOILY,10,2011-10-13 12:50:00+00:00,1.65,12446,RSA
4,571035,23201,JUMBO BAG ALPHABET,10,2011-10-13 12:50:00+00:00,2.08,12446,RSA


In [18]:
df_bigQuery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406829 entries, 0 to 406828
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   InvoiceNo    406829 non-null  object             
 1   StockCode    406829 non-null  object             
 2   Description  406829 non-null  object             
 3   Quantity     406829 non-null  Int64              
 4   InvoiceDate  406829 non-null  datetime64[ns, UTC]
 5   UnitPrice    406829 non-null  float64            
 6   CustomerID   406829 non-null  Int64              
 7   Country      406829 non-null  object             
dtypes: Int64(2), datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 25.6+ MB
